# 입문자를 위한, 파이썬/R 데이터 분석  

]

## Today's mission

- selenium 패키지 활용 데이터 수집
- 실습 (1) 한국거래소 기업공시채널 상장법인 목록 읽기
- 실습 (2) 주식 종가 차트 그리기
- 실습 (3) 네이버 주식정보 가져오기

]

##### 패키지 Import 방법 참조 :
- 1) `import pkg_name` : pkg_name 이름의 패키지를 로딩합니다 (일반적 방법)
- 2) `import pkg_name.sub` : pkg_name 와 하위 패키지 sub 까지 로딩합니다 (하위 패키지까지 로딩할 경우)
- 3) `from pkg_name.sub import abc` : pkg_name와 하위 패키지 sub 에다가 abc 모듈 또는 함수까지 로딩합니다

## 실습 (1) 한국거래소 기업공시채널 상장법인 목록 읽기

#### 참고 :  https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage

![KIND](data/kind.jpg)

#### 한국거래소 기업공시채널(kind.krx.co.kr) > 상장법인상세정보 > 상장법인목록 > EXCEL 다운로드

![KIND](data/kind_excel.jpg)

#### 그런데, EXCEL 파일 같지만 메모장으로 열어보면 HTML 형태 !!!

![KIND](data/kind_notepad.jpg)

#### 그러므로, 판다스 read_html()로 데이터를 읽어야함

In [ ]:
#!pip install html5lib
#!pip install lxml

In [ ]:
import pandas as pd
krx_list = pd.read_html("data/상장법인목록.xls")

In [ ]:
krx_list[0]#.info()

In [ ]:
krx_list[0].종목코드 = krx_list[0].종목코드.map('{:06d}'.format)

In [ ]:
krx_list[0].info()

In [ ]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]

- (중요) API 서버의 Parameter를 어떻게 찾을까?
- https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage
- [Chrome]-<개발자도구 F12>-<Network>-<Excel 다운로드 클릭>-[다운로드 파일 Name/Payload 분석]
- REST API 및 GET, PUT 방식 : https://studyandwrite.tistory.com/453

In [ ]:
df

In [ ]:
df['종목코드'] = df['종목코드'].map('{:06d}'.format)

In [ ]:
df = df.sort_values(by='종목코드')

In [ ]:
df

## 실습 (2) 주식 종가 차트 그리기

- xmltodict 라이브리리

In [ ]:
#!pip install mplfinance
#!pip install xmltodict

In [ ]:
##importing libraries
from urllib.request import urlopen
import numpy as np
import lxml
import requests
import json
import xmltodict
import mplfinance as mpf
import pandas as pd

In [ ]:
#data crawling using json
url='https://fchart.stock.naver.com/sise.nhn?symbol=005930&timeframe=day&count=200&requestType=0'
rs=requests.get(url)
dt=xmltodict.parse(rs.text)
js=json.dumps(dt)
js=json.loads(js)

In [ ]:
js

In [ ]:
data=pd.json_normalize(js['protocol']['chartdata']['item'])
data

In [ ]:
df=data['@data'].str.split('|', expand=True)
df.columns=['Date','Open','High','Low','Close','Volume']

In [ ]:
df

In [ ]:
#data handling
df['Open']=pd.to_numeric(df['Open'])
df['High']=pd.to_numeric(df['High'])
df['Low']=pd.to_numeric(df['Low'])
df['Close']=pd.to_numeric(df['Close'])
df['Volume']=pd.to_numeric(df['Volume'])
df_final=df[['Open','High','Low','Close','Volume']]
df_final_time=pd.DatetimeIndex(df['Date'])
df_final.index=df_final_time

In [ ]:
df_final

In [ ]:
#Visualization
kwargs=dict(title="Samsung", type='candle',mav=(5,20,60), volume=True)
mc=mpf.make_marketcolors(up='red', down='blue', inherit=True)
style_final=mpf.make_mpf_style(marketcolors=mc)
mpf.plot(df_final, **kwargs, style=style_final)
#plt.show()

## 실습 (3) 네이버 주식정보 가져오기

#### 사이트 : https://finance.naver.com/sise/sise_market_sum.nhn
#### 참고 : https://cocoabba.tistory.com/20

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://finance.naver.com/sise/sise_market_sum.nhn?page=3"
res = requests.get(url)
res.text

In [ ]:
url

In [ ]:
soup = BeautifulSoup(res.text, 'lxml')
print(soup)

In [ ]:
stock_head = soup.find("thead").find_all("th")
stock_head

In [ ]:
data_head = [head.get_text() for head in stock_head]
data_head

In [ ]:
stock_list = soup.find("tbody").find_all("tr")
#stock_list
stock_list[1].get_text().split()

In [ ]:
for stock in stock_list:
     if len(stock) > 1 :
          print(stock.get_text().split())